In [1]:
from neo4j import GraphDatabase
import pandas as pd

driver  = GraphDatabase.driver(uri ="bolt://localhost:7687", auth =("neo4j","test"))
session = driver.session()
query1 = "Match (x:hbf),(y:hbf) where not(x.name = y.name or y.name = x.name) 
WITH point({longitude:toFloat(x.hbf_longitude),latitude:toFloat(x.hbf_latitude)}) AS p1, 
point({longitude:toFloat(y.hbf_longitude),latitude:toFloat(y.hbf_latitude)}) AS p2, 
x.name as city1, y.name as city2 WITH toFloat(round(distance(p1,p2))*0.001)  AS Distance, 
city1 as Source, city2 as Destination 
RETURN Source, Destination, Distance"
 
records = session.run(query1)

d=[]

#df=pd.DataFrame()
for record in records:
    d.append(record)
df=pd.DataFrame(d)

j=0
#print(type(df.iloc[45,2]))
#dic=dict()

for val in df[2]:
    i = float(val)
    if(i!=0):
        query2 = "Match (x:hbf),(y:hbf)where x.name=$vname1 and y.name=$vname2 
        Create (x)-[r:train{dist:$vname3}]->(y)"
        dic["vname1"] = df.iloc[j,0]
        dic["vname2"] = df.iloc[j,1]
        dic["vname3"] = df.iloc[j,2]
        #dic["vname4"] = df.iloc[j,3]
        session.run(query2,dic)        
    j=j+1
print("Done")
